## Briefly


### __ Problem Statement __
- Obtain news from google news articles
- Sammarize the articles within 60 words
- Obtain keywords from the articles










##### Importing all the necessary libraries required to run the following code 

In [1]:
from gnewsclient import gnewsclient   # for fetching google news
from newspaper import Article         # to obtain text from news articles
from transformers import pipeline     # to summarize text
import spacy                          # for named entity recognition

##### Load sshleifer/distilbart-cnn-12-6 model

In [2]:
def load_model():                           
    model = pipeline('summarization')
    return model
data = gnewsclient.NewsClient(max_results=0)
nlp = spacy.load("en_core_web_lg") 

##### Obtain urls and it's content

In [3]:
def getNews(topic,location):                
    count=0
    contents=[]
    titles=[]
    authors=[]
    urls=[]
    data = gnewsclient.NewsClient(language='english',location=location,topic=topic,max_results=10) 
    news = data.get_news()  
    for item in news:
        url=item['link']
        article = Article(url)
        try:
            article.download()
            article.parse()
            temp=item['title'][::-1]
            index=temp.find("-")
            temp=temp[:index-1][::-1]
            urls.append(url)
            contents.append(article.text)
            titles.append(item['title'][:-index-1])    
            authors.append(temp)
            count+=1
            if(count==5):
                break
        except:
            continue 
    return contents,titles,authors,urls         

##### Summarizes the content- minimum word limit 30 and maximum 60

In [4]:
def getNewsSummary(contents,summarizer):   
    summaries=[]     
    for content in contents:
        minimum=len(content.split())
        summaries.append(summarizer(content,max_length=60,min_length=min(30,minimum),do_sample=False,truncation=True)[0]['summary_text'])   
    return summaries

##### Named Entity Recognition

In [5]:
# Obtain 4 keywords from content (person,organisation or geopolitical entity) 
def generateKeyword(contents):            
    keywords=[]
    words=[]      
    labels=["PERSON","ORG","GPE"]
    for content in contents:
        doc=nlp(content)
        keys=[]
        limit=0
        for ent in doc.ents:
            key=ent.text.upper()
            label=ent.label_
            if(key not in words and key not in keywords and label in labels): 
                keys.append(key)
                limit+=1
                for element in key.split():
                    words.append(element)
            if(limit==4):
                keywords.append(keys)
                break                           
    return keywords
  

##### Displaying keywords 

In [6]:
def printKeywords(keywords):
    for keyword in keywords:
        print(keyword)

##### Displaying the Summary with keywords in it highlighted

In [7]:
def printSummary(summaries,titles):
    for summary,title in zip(summaries,titles):
        print(title.upper(),'\n')
        print(summary)
        print("\n\n")

In [8]:
summarizer=load_model() 

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [9]:
contents,titles,authors,urls=getNews("Sports","India")

In [10]:
summaries=getNewsSummary(contents,summarizer)

In [11]:
keywords=generateKeyword(contents)

In [12]:
printKeywords(keywords)

['INDIA', 'SCOTLAND', 'SUPER 12', 'DUBAI']
["VIRAT KOHLI'S", 'TEAM INDIA', 'DHONI', 'UAE']
['AUSTRALIA', 'AFGHANISTAN', 'CRICKET AUSTRALIA', 'CRICBUZZ STAFF •']
['GARY STEAD', 'TRENT BOULT', 'COLIN DE GRANDHOMME', 'BLACKCAPS']
['DWAYNE BRAVO', 'SRI LANKA', 'ICC', 'THE WEST INDIES']


In [13]:
printSummary(summaries,titles)

T20 WORLD CUP 2021, IND VS SCO PREVIEW: INDIA FACE SCOTLAND, EYE ANOTHER BIG WIN  

 India take on Scotland in a Super 12 clash of the 2021 T20 World Cup in Dubai on Friday . Virat Kohli-led side beat Afghanistan by 66 runs in Abu Dhabi on Wednesday . India must win their remaining two games while maintaining high run rates and hope for New Zealand to



‘THERE ARE MANY CANDIDATES BUT HE’S THE BEST': SEHWAG PICKS NEXT INDIA CAPTAIN AFTER KOHLI STEPS DOWN AT END OF T20 WC  

 Virat Kohli set to step down as T20I captain after this World Cup in UAE and Oman . Many experts are anticipating his deputy Rohit Sharma to fill up the position . Former India opener Virender Sehwag backed Rohit as the ideal candidate .



ONE-OFF TEST VS AFGHANISTAN POSTPONED, CONFIRMS CRICKET AUSTRALIA | CRICBUZZ.COM - CRICBUZZ  

 Cricket Australia's one-off Test against Afghanistan has officially been postponed . The historic Test has been hanging in the balance since the CA revealed that they wouldn't support